In [1]:
import os
import gym
import numpy as np

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.callbacks import BaseCallback

import custom_tanks_disc_mult
from stable_baselines3.common.vec_env import SubprocVecEnv

from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3 import PPO

In [2]:
torch.backends.cudnn.benchmark = True

## Callback class

In [3]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model_PPO')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        pass
        # Create folder if needed
#         if self.save_path is not None:
#             os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose > 0:
                print("Num timesteps: {}".format(self.num_timesteps))
                print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose > 0:
                    print("Saving new best model to {}".format(self.save_path))
                  self.model.save(self.save_path)

        return True

## Neural network class

In [4]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """

    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 518):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        
        
        extractors = {}
        
        for key, subspace in observation_space.spaces.items():
            if key == "img":
        
                n_input_channels = observation_space[key].shape[0]
            
                extractors[key] = nn.Sequential(

                nn.Conv2d(n_input_channels, 32, 2),
                nn.MaxPool2d(2, 2),
                nn.Conv2d(32, 64, 2),
                nn.MaxPool2d(2, 2),

                # ResBlock(n_filters=64, kernel_size=2),
                # nn.MaxPool2d(4, 4),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2),
                nn.MaxPool2d(2, 2),
                ResBlock(n_filters=64, kernel_size=2), 
                nn.MaxPool2d(2, 2),
                
                nn.Conv2d(64, 128, 2),
                nn.Flatten())
                    
            elif key == "posRobot":
                
                n_input_channels = observation_space[key].shape[0]
                
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                    
            elif key == "target":
                            
                n_input_channels = observation_space[key].shape[0]
                    
                extractors[key] = nn.Sequential(nn.Linear(n_input_channels, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 9),
                                        nn.ReLU(),
                                        nn.Linear(9, 3))
                
        self.extractors = nn.ModuleDict(extractors)

    def forward(self, observations: th.Tensor) -> th.Tensor:
        '''
        Forward propagation
        :param observations: (dict) изображение; координаты и углы ориентации агентов
        :return: features tensor
        '''
        encoded_tensor_list = []

        for key, extractor in self.extractors.items():
            encoded_tensor_list.append(extractor(observations[key]))

        return th.cat(encoded_tensor_list, dim=1)

    
class ResBlock(nn.Module):
    def __init__(self, n_filters, kernel_size):
        """
        Инициализация кастомного резнетовского блока
        :param n_filters: (int) количество фильтров сверточного слоя
        :param kernel_size: (int) размер ядра свертки
        """
        super().__init__()
        self.n_filters = n_filters
        self.kernel_size = kernel_size

        self.b1 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
    
        self.b2 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        self.b3 = nn.Conv2d(self.n_filters, self.n_filters, self.kernel_size, padding='same')
        self.b4 = nn.BatchNorm2d(self.n_filters, eps = 0.001, momentum= 0.99)
        
    def forward(self, x):
        '''
        Forward propagation
        :param x: input
        :return: output
        '''
        residual = x
        y = F.relu(self.b1(x))
        y = self.b2(y)
        y = F.relu(self.b3(y))
        y = self.b4(y)
        y += residual
        y = F.relu(y)
        return y

## Make environment

In [5]:
def make_env(env_id, rank, seed=0, **kwargs):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id, **kwargs)
        env.seed(seed + rank)
        log_dir = './saved_models/PPO/callback' + str(rank) + '/'  # For A2C agent: './saved_models/A2C/callback'
        os.makedirs(log_dir, exist_ok=True)
        env = Monitor(env, log_dir)
        return env
    set_random_seed(seed)
    
    return _init

In [6]:
env_id = 'CustomTanks_disc_mult-v0'
num_procs = 5
env = SubprocVecEnv([make_env(env_id=env_id,
                              rank=i,
                              vizualaze     = False,
                              obstacle_turn = False,
                              Total_war     = True,
                              head_velocity = 0.005,
                              num_obs       = 5,
                              num_enemy     = 1,
                              size_obs      = [30, 40],
                              rew_col       = -100,
                              rew_win       = 100, 
                              rew_defeat    = -100,
                              steps_limit   = 2000) for i in range(num_procs)], start_method='fork')

pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)Hello from the pygame community. https://www.pygame.org/contribute.html

Hello from the pygame community. https://www.pygame.org/contribute.html
pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/parkhom/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/parkhom/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/parkhom/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/parkhom/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/parkhom/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


## Initialize the agent

In [7]:
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=518),
    activation_fn=torch.nn.ReLU,
    net_arch = [dict(pi=[1029, 128, 32, 8], vf=[1029, 128, 32, 8])])

model = PPO(policy          = 'MultiInputPolicy',
            env             = env,
            learning_rate   = 0.0001,
            n_steps         = 125,
            batch_size      = 5,
            gamma           = 0.99,
            gae_lambda      = 0.95,
            tensorboard_log = "./tensorboard_logs/",
            policy_kwargs   = policy_kwargs,
            verbose         = 2,
            device          = 'cuda:0')

# model = A2C(policy          = 'MlpPolicy',
#             env             = env,
#             learning_rate   = 0.0001,
#             n_steps         = 24,
#             gamma           = 0.99,
#             gae_lambda      = 0.95,
#             tensorboard_log = "./tensorboard_logs/",
#             policy_kwargs   = policy_kwargs,
#             verbose         = 0,
#             device          = 'cuda')

Using cuda:0 device
Wrapping the env in a VecTransposeImage.


## Make callback

In [8]:
log_dir = './saved_models/PPO/callback0/'  # For A2C agent: './saved_models/A2C/callback'
callback = SaveOnBestTrainingRewardCallback(check_freq=5000, log_dir=log_dir)

## Learn the model

In [ ]:
model.learn(total_timesteps=1e6,callback=callback)

Logging to ./tensorboard_logs/PPO_1


/home/parkhom/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:453: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:882.)
  return F.conv2d(input, weight, bias, self.stride,


----------------------------
| time/              |     |
|    fps             | 114 |
|    iterations      | 1   |
|    time_elapsed    | 5   |
|    total_timesteps | 625 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 28          |
|    iterations           | 2           |
|    time_elapsed         | 43          |
|    total_timesteps      | 1250        |
| train/                  |             |
|    approx_kl            | 0.016227538 |
|    clip_fraction        | 0.224       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.06       |
|    explained_variance   | -0.00438    |
|    learning_rate        | 0.0001      |
|    loss                 | 29.1        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0242     |
|    value_loss           | 219         |
-----------------------------------------
-----------------------------------------

## Make environment and load the best model to test trained model and get statistics

In [ ]:
path = './saved_models/PPO/callback0/best_model_PPO/'  # For A2C agent: './saved_models/A2C/callback0/best_model_A2C/'
model = PPO.load(path, env=env_test)  # For A2C agent: A2C.load(path, env=env_test)

In [ ]:
env.get_statistic(model, 1000)

## Check tensorboard

In [ ]:
# !tensorboard --logdir ./tensorboard_logs/